#### Importing all the required packages
<a href="https://github.com/curiousily/Get-Things-Done-with-Prompt-Engineering-and-LangChain/blob/master/01.loaders-indexes-vectorstores.ipynb">reference link</a>

In [1]:
import json
import os
import time
import textwrap
import pickle
import chromadb
import langchain
import openai

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import WebBaseLoader, url_selenium, unstructured
from langchain.embeddings import OpenAIEmbeddings
from langchain.indexes import VectorstoreIndexCreator
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from OpenAI_VectorDb_experiment.__init__ import OPENAI_API_KEY

import copy
import re


#### Setting environment key OPENAI API Key

In [2]:
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Loading all the urls of Fission Labs using <b>Selenium Loader</b>, which was better than <b>unstructured</b> and <b>WebBaseLoader</b>.
Please feel free to experiment with other tools available as well for better extraction of data

In [3]:
loader = url_selenium.SeleniumURLLoader([
    "http://fissionlabs.com/about-us",
    "http://fissionlabs.com/how-we-work",
    "http://fissionlabs.com/public-relations-csr",
    "http://fissionlabs.com/careers",
    "http://fissionlabs.com/contact-us",
    "http://fissionlabs.com/services/web-mobile-application-development-services",
    "http://fissionlabs.com/services/data-engineering-services",
    "http://fissionlabs.com/services/cloud-consulting-services",
    "http://fissionlabs.com/services/ai-ml-based-solutions",
    "http://fissionlabs.com/services/quality-assurance-services",
    "http://fissionlabs.com/services/salesforce-consulting-services",
    "http://fissionlabs.com/blog",
    # "http://fissionlabs.com/case-studies",
    "https://www.fissionlabs.com/case-study/hospital-management-analytics-platform",
    "http://fissionlabs.com/case-study/ai-based-object-volume-estimator",
    "http://fissionlabs.com/case-study/multi-device-iot-communication-platform",
    "http://fissionlabs.com/case-study/realtime-health-records-monitoring-platform",
    "http://fissionlabs.com/case-study/fully-automated-managed-cloud-services-platform-for-healthcare-industry",
    "http://fissionlabs.com/case-study/advertising-sales-automation-workflow-platform"
    "https://www.fissionlabs.com/case-study/fleet-tracking-portal",
    "https://www.fissionlabs.com/case-study/pathology-radiology-e-learning-platform",
    "https://www.fissionlabs.com/case-study/enterprise-hrms-labour-analysis-platform",
    "http://fissionlabs.com/case-studies/integrated-campaign-management-platform",
    "http://fissionlabs.com/case-studies/workflow-management-platform",
    "http://fissionlabs.com/case-studies/iot-driven-fleet-management-platform",
    "http://fissionlabs.com/case-studies/life-science-and-regulatory-platform",
    "http://fissionlabs.com/case-studies/integrated-telehealth-platform",
    "http://fissionlabs.com/e-book-whitepapers",
    "http://fissionlabs.com/blog-posts/selecting-the-right-software-development-vendor-how-proof-of-concept-trials-can-help",
],
    browser="chrome"
)

#### We have an issue with case studies page so we are using WebBaseLoader

In [5]:
loader1 = WebBaseLoader(["http://fissionlabs.com/case-studies"])
temp = loader1.load()

#### Extracting all the document using loader

In [6]:
documents = loader.load()
documents.extend(temp)
len(documents)

28

#### Url Information is available in metadata

In [7]:
documents[0].metadata

{'source': 'http://fissionlabs.com/about-us'}

In [8]:
documents[0].page_content[:200]

'Company\n\nAbout Us\n\nHow We Work\n\nPublic Relations & CSR\n\nCareers\n\nContact Us\n\nServices\n\nWeb & Mobile App Development\n\nData Engineering Services\n\nCloud Consulting Services\n\nAI & ML Based Solutions\n\nQual'

chunking to make it more efficient <b>2048-8192 for <a href="https://www.marktechpost.com/2022/12/17/openai-announce-a-new-embedding-model-which-is-significantly-more-capable-cost-effective-and-simpler-to-use/">OPENAI</a> </b>
Please feel free to fine-tune and experiment

In [7]:
text_splitter = CharacterTextSplitter(
    separator = " ",
    chunk_size = 3000,
    chunk_overlap  = 500
)

#### Creating sub documents
Feel free to experiment with text_splitter and etc

In [9]:
docs = []
for index, doc in enumerate(documents):
    temp = copy.deepcopy(doc)

    ## cleaning if you like
    # content = temp.page_content
    # clean_content = " ".join(content.split("\n\n")).strip()
    # clean_content = clean_content.replace(".\n",".")
    ## replacing the un-necessary content which is common across all documents
    # clean_content = clean_content.replace(" E-Books & Whitepapers Latest Blog Innovating with Purpose: Inside Fission Labs Dynamic Work Approach Case Studies Thank you! Your submission has been received! Oops! Something went wrong while submitting the form.",":")
    # clean_content = clean_content.replace("All Rights Reserved Fission Labs uses cookies to improve functionality, performance and effectiveness of our communications. By continuing to use this site, or by clicking “I agree” you consent to the use of cookies. Detailed information on the use of cookies is provided on our  Cookies Policy” Got it","")
    # clean_content = clean_content.replace('© 2023 Fission Labs','')
    # clean_content = clean_content.strip()
    # clean_content = clean_content.replace('\' ',' ')
    # temp.page_content = clean_content
    docs.append(temp)

    ## ----- text_splitter please feel free to experiment with only the number of documents would increase -----
    # output = text_splitter.create_documents([temp.page_content], [temp.metadata])
    # for o in output:
    #     if o is not None or o.page_content is not None:
    #         docs.append(o)
    # docs.append(temp)

#### Actual documents vs seperated docs with overlap of 200 tokens

In [10]:
len(documents)

28

In [11]:
len(docs)

28

In [11]:
docs[0]

Document(page_content='Company\n\nAbout Us\n\nHow We Work\n\nPublic Relations & CSR\n\nCareers\n\nContact Us\n\nServices\n\nWeb & Mobile App Development\n\nData Engineering Services\n\nCloud Consulting Services\n\nAI & ML Based Solutions\n\nQuality Assurance Services\n\nSalesforce Consulting Services\n\nResources\n\nBlogs\n\nCase Studies\n\nE-Books & Whitepapers\n\nLatest Blog\n\nInnovating with Purpose: Inside Fission Labs\' Dynamic Work Approach\n\nAbout Us\n\nCreating Flexible, Agile, and Scalable Solutions is in Our DNA\n\nWith a humble beginning in \n\n2008, Fission Labs was born with a simple aim to leverage our engineering capabilities and help Startups overcome technological challenges.\n\nAfter a decade, Fission Labs is a family of over 350+ seasoned engineers with product development at heart and years of experience developing custom solutions and pioneering digital transformation across various technology verticals, including AI/ML, Data Engineering, and Cloud solutions.\n\n

In [12]:
len(docs[0].page_content)

5760

#### Experimenting using vector database
chroma vector database using OPENAI Key

In [11]:
index = VectorstoreIndexCreator().from_loaders([loader])

Using embedded DuckDB without persistence: data will be transient


##### Querying on our index
Please feel free to experiment on more questions

In [14]:
query = """
You're Dwight K. Schrute from the Office.
What are QA services in FissionLabs?
"""
index.query(query)

' Fission Labs QA services helps you optimize your application or platform performance no matter what stage of the product lifecycle you’re at. We provide quality assurance services to a variety of clients—ranging from small Startups to large corporations. We help our clients build better products and services through a variety of different techniques, including human testing, automation, and review of product code. We’re proud to say that we’ve helped our clients make some of the best products and services in their respective industries. Well versed in the use of the latest tools in integration, black box, regression, and automation, our rigorous software testing services and QA ensures that your applications and platform perform flawlessly. We also provide other QA services to fit a variety of needs. Our team can help you build, implement, and scale your testing processes, or design a quality assurance strategy for your product or service. We’re here to help you build a better QA tea

Only it was able to <b>retrieve 4/14</b>

In [15]:
query = """
You're Dwight K. Schrute from the Office.

What are the case studies available?
"""
index.query(query)

' The case studies available are Multi Device IoT Communication Platform, Hospital Management & Analytics Platform, Enterprise HRMS & Labour Analysis Platform, and Integrated Campaign Management Platform.'

In [13]:
model = OpenAI(temperature=0)
embeddings = OpenAIEmbeddings()

In [14]:
# db = Chroma(embedding_function=embeddings).from_documents(docs, embeddings)
db = Chroma.from_documents(docs, embeddings)

Using embedded DuckDB without persistence: data will be transient


#### Just feel free to experiment with

In [18]:
query = """
You're Dwight K. Schrute from the Office.

What are the case studies available?
"""
db.similarity_search(query,k=2)

[Document(page_content="Company\n\nAbout Us\n\nHow We Work\n\nPublic Relations & CSR\n\nCareers\n\nContact Us\n\nServices\n\nWeb & Mobile App Development\n\nData Engineering Services\n\nCloud Consulting Services\n\nAI & ML Based Solutions\n\nQuality Assurance Services\n\nSalesforce Consulting Services\n\nResources\n\nBlogs\n\nCase Studies\n\nE-Books & Whitepapers\n\nLatest Blog\n\nInnovating with Purpose: Inside Fission Labs' Dynamic Work Approach\n\nThank you! Your submission has been received!\n\nView file in another tab\n\nOops! Something went wrong while submitting the form.\n\nCase Study\n\nHospital Management & Analytics Platform\n\nDownload Case study\n\nBusiness Challenge\n\nOur client collaborates with top oncology experts, cancer research, and treatment institutions and sought Fission Labs' technical aid for the following tasks: developing an admin application with client/domain/avatar/tenants and associated permissions, establishing and overseeing permissions for multiple us

#### Just checking a sample embeddings

In [17]:
openai_embedding = embeddings.embed_documents([docs[0].page_content])
len(openai_embedding[0])

1536

In [18]:
openai_embedding[0][:5]

[0.021486673037638765,
 0.007945232852165873,
 -0.008159409100000431,
 -0.013969792681142009,
 -0.012933457920802001]

In [19]:
db.get().keys()

dict_keys(['ids', 'embeddings', 'documents', 'metadatas'])

In [20]:
db.get()["ids"][:5]

['56b18f36-35c1-11ee-8e6f-acde48001122',
 '56b196a2-35c1-11ee-8e6f-acde48001122',
 '56b1909e-35c1-11ee-8e6f-acde48001122',
 '56b190e4-35c1-11ee-8e6f-acde48001122',
 '56b1912a-35c1-11ee-8e6f-acde48001122']

In [21]:
db.get()["documents"][0][:50]

'Company\n\nAbout Us\n\nHow We Work\n\nPublic Relations &'

In [15]:
template_exp = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}

Please the provide the following in json JSON format with following keys ANSWER and source url."""
prompt = PromptTemplate(template=template_exp, input_variables=["context", "question"])

In [16]:
chain_type_kwargs = {"prompt": prompt}
chain = RetrievalQA.from_chain_type(
    # llm=model,
    llm = ChatOpenAI(temperature=0),
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
)


In [17]:
def get_output(question, chain=chain):
    response = chain.run(question)
    output = json.loads(response)
    return output

In [18]:
resp = get_output("Who is Surendra Mallampati?")
resp

{'ANSWER': 'Surendra Mallampati is a member of the leadership team at Fission Labs and has over 20 years of industry experience. He has played a key role in pioneering digital transformation across various technology verticals, including AI/ML, Data Engineering, and Cloud solutions.',
 'source url': 'https://www.fissionlabs.com/about-us/'}

In [19]:
resp = get_output("Who is CEO of Apple?")
resp

{'ANSWER': "I don't know", 'source url': 'N/A'}

In [20]:
resp = get_output("When was Fission labs started?")
resp

{'ANSWER': 'Fission Labs was started in 2008.',
 'source url': 'https://www.fissionlabs.com/about-us/'}

In [21]:
response = get_output("Delivery models")
response

{'ANSWER': 'The delivery models offered by Fission Labs are Milestone Delivery, Agile Delivery, and Rapid Application Dev.',
 'source url': 'https://www.fissionlabs.com/how-we-work/'}

##### Please join and send if it's a list

In [22]:
query = "What are the core values at Fission Labs?"
response = get_output(query)
response


{'ANSWER': ['Integrity',
  'Accountability',
  'Passion',
  'Excellence',
  'Teamwork'],
 'source url': 'https://www.fissionlabs.com/about-us/'}

##### Please check the output format and convert into HTML format appropriate for dictionary

In [23]:
query = "core values"
response = get_output(query)
response

{'ANSWER': {'Integrity': 'Integrity is non-negotiable at Fission Labs. Therefore, we stick to the highest levels of ethics and standards and exhibit the same on professional and personal levels.',
  'Accountability': "We hold ourselves accountable to drive success for our customers, investors, and each other. We believe it's the backbone of our success.",
  'Passion': "Passion is stitched in each of our members' hearts and allows us to walk an extra mile to drive excellence.",
  'Excellence': 'We relentlessly pursue the next level of excellence by holding ourselves to the highest levels of superiority and making continuous improvement with each accomplishment.',
  'Teamwork': 'Fostering teamwork is about creating a culture of collaboration. We believe the sum of all our efforts together is bigger than any individual accomplishments.'},
 'source url': 'https://www.fissionlabs.com/about-us/'}

In [24]:
query = "What is Product Development?"
response = get_output(query)
response

{'ANSWER': 'Product development is the process of creating and improving products or services through research, design, and testing.',
 'source url': 'https://www.fissionlabs.com/how-we-work/'}

In [25]:
query = "Product Development"
response = get_output(query)
response

{'ANSWER': "Proof of concept trials are a small-scale implementation of a software project or solution to evaluate its feasibility, functionality, and suitability for a specific business requirement. They allow you to assess a vendor's technical competence, test compatibility with your infrastructure, evaluate user experience, mitigate risks, and build trust and collaboration. Fission Labs offers 2 Weeks Free Proof of Concept (POC) Trial Services to help clients evaluate their technical expertise and suitability for their project.",
 'source_url': 'https://www.fissionlabs.com/blog/selecting-the-right-software-development-vendor-how-proof-of-concept-trials-can-help/'}

In [51]:
response = get_output("Product development team")
response

{'ANSWER': 'The product development team at Fission Labs consists of various roles including UX/UI Designer, Developer, DevOps Engineer, Quality Engineer, Product Manager, Data Engineer, Business Analyst, and Scrum Master.',
 'source url': 'https://www.fissionlabs.com/how-we-work/'}

In [26]:
query = "quality automation"
response = get_output(query)
response

{'ANSWER': 'Fission Labs provides quality assurance services that include automation testing. They use the latest tools and technology available in the market, including script-less automation frameworks. They offer automation for regression suite testing and continuous testing, as well as automation for API, web service, and microservice testing. Their quality assurance services also cover functional and non-functional testing, such as system testing, integration testing, performance testing, scalability testing, usability testing, load testing, security testing, and mobile testing. Fission Labs aims to accelerate time-to-market and improve reliability through their automation services.',
 'source url': 'https://www.fissionlabs.com/quality-assurance-services/'}

In [27]:
query = "automation"
response = get_output(query)
response

{'ANSWER': "Fission Labs' engineering team delivered a scalable, intelligent, and data-driven fleet management platform to deliver peak efficiency.",
 'source url': 'https://www.fissionlabs.com/case-studies/iot-driven-fleet-management-platform'}

In [42]:
query = "qa services"
response = get_output(query)
response


{'ANSWER': 'Fission Labs provides quality assurance services to a variety of clients, ranging from small startups to large corporations. They offer services for every stage of the product lifecycle and use the latest tools and technology in integration, black box, regression, and automation testing. They also provide services for functional and non-functional testing, mobile testing, and data testing. Fission Labs aims to help their clients build better products and achieve their goals through their quality assurance services.',
 'source url': 'https://www.fissionlabs.com/quality-assurance-services/'}

In [52]:
query = "case studies"
response = get_output(query)
response

{'ANSWER': 'Case studies are available on the Fission Labs website.',
 'source url': 'https://www.fissionlabs.com/case-studies/'}

In [53]:
query = "case studies available"
response = get_output(query)
response

{'ANSWER': 'Case studies are available on the Fission Labs website.',
 'source url': 'https://www.fissionlabs.com/case-studies'}

In [28]:
query = "What are the case studies available?"
response = get_output(query)
response

{'ANSWER': ['Hospital Management & Analytics Platform',
  'AI Based Object Volume Estimator',
  'Multi Device IoT Communication Platform',
  'Realtime Health Records Monitoring Platform',
  'Fully Automated Managed Cloud Services Platform For Healthcare Industry',
  'Advertising Sales Automation & Workflow Platform',
  'Fleet Tracking Portal',
  'Pathology & Radiology E‑Learning Platform',
  'Enterprise HRMS & Labour Analysis Platform',
  'Integrated Campaign Management Platform',
  'IOT-Driven Fleet Management Platform',
  'Workflow Management Platform',
  'Life Science Regulatory Platform',
  'Integrated TeleHealth Platform'],
 'source url': 'https://www.fissionlabs.com/case-studies/'}

In [29]:
query = "What are the projects available?"
response = get_output(query)
response

{'ANSWER': ['Hospital Management & Analytics Platform',
  'AI Based Object Volume Estimator',
  'Multi Device IoT Communication Platform',
  'Realtime Health Records Monitoring Platform',
  'Fully Automated Managed Cloud Services Platform For Healthcare Industry',
  'Advertising Sales Automation & Workflow Platform',
  'Fleet Tracking Portal',
  'Pathology & Radiology E‑Learning Platform',
  'Enterprise HRMS & Labour Analysis Platform',
  'Integrated Campaign Management Platform',
  'IOT-Driven Fleet Management Platform',
  'Workflow Management Platform',
  'Life Science Regulatory Platform',
  'Integrated TeleHealth Platform'],
 'source url': 'https://www.fissionlabs.com/case-studies/'}

In [39]:
query = "projects available"
response = get_output(query)
response

{'ANSWER': 'Hospital Management & Analytics Platform',
 'source url': 'https://www.fissionlabs.com/case-studies/hospital-management-analytics-platform'}

In [31]:
get_output("about fission labs?")

{'ANSWER': 'Fission Labs is a company that provides cloud consulting services and other comprehensive development services. They help businesses manage technology challenges and take advantage of new opportunities in the cloud. They offer a wide range of cloud consulting services, including cloud strategy design, cloud service implementation, cloud infrastructure scaling, and cloud migration. They use advanced tools and technology, and are registered and certified partners with cloud solution providers like AWS, Azure, and GCP. They also offer services in data engineering, AI and ML solutions, web and mobile app development, quality assurance, and Salesforce consulting. They have a project implementation approach that includes digital workshops, rapid prototyping, and productionization. They offer different delivery models to fit unique needs, including milestone delivery, agile delivery, and rapid application development. They have received positive feedback from clients for their off

In [51]:
get_output("Who is the 6th Leadership member of Fission Labs?")

{'ANSWER': 'There is no information provided about the 6th Leadership member of Fission Labs.',
 'source url': 'N/A'}

#### About case studies

In [32]:
get_output("What can you tell me about Fleet Management Portal case study?")

{'ANSWER': 'The Fleet Management Portal case study is about a leading logistic service provider that wanted to improve their operations with technological implementations. Fission Labs delivered a scalable, intelligent, and data-driven fleet management platform that improved prediction of pickup and delivery time accuracy, reduced costs due to idling and resource underutilization, decreased idle time between trips, reduced fuel theft and pilferage, and improved trip efficiency. The platform features capturing and storing sensor data, automating driver and vehicle pairings, and ensuring accountability for drivers and trips.',
 'source url': 'https://www.fissionlabs.com/case-studies/iot-driven-fleet-management-platform'}

In [33]:
get_output("Can you explain me about Fleet tracking Portal case study?")

{'ANSWER': 'The Fleet tracking Portal case study is about a leading logistic service provider that wanted to improve the efficiency of their operations. Fission Labs developed a scalable and data-driven fleet management platform for them. The platform improved the accuracy of pickup and delivery time predictions, reduced costs due to idling and resource underutilization, minimized idle time between trips, decreased fuel theft and pilferage, and ensured peak vehicle health to avoid breakdowns. The platform features capturing and storing sensor data, automating driver and vehicle pairings, and ensuring accountability for trips. For more information, you can refer to the case study at: https://www.fissionlabs.com/case-studies/iot-driven-fleet-management-platform',
 'source url': 'https://www.fissionlabs.com/case-studies/iot-driven-fleet-management-platform'}

In [34]:
get_output("What's the business challenge in Fleet Tracking?")

{'ANSWER': 'The business challenge in Fleet Tracking is to revolutionize the industry with disruptive technological implementations for faster and more efficient operations.',
 'source url': 'https://www.fissionlabs.com/case-studies/iot-driven-fleet-management-platform'}

#### Services

In [35]:
get_output("services")


{'ANSWER': 'Fission Labs provides a range of services including Web & Mobile App Development, Data Engineering Services, Cloud Consulting Services, AI & ML Based Solutions, Quality Assurance Services, and Salesforce Consulting Services.',
 'source url': 'https://www.fissionlabs.com/services/'}

In [37]:
get_output("What services do you offer at Fission Labs?")


{'ANSWER': 'Fission Labs offers a variety of services including web and mobile app development, data engineering services, cloud consulting services, AI and ML based solutions, quality assurance services, and Salesforce consulting services.',
 'source': 'https://www.fissionlabs.com/services/'}

In [38]:
get_output("What are the web and mobile bases services you offer?")

{'ANSWER': 'The web and mobile based services offered by Fission Labs include web and mobile application development, consulting services, digital transformation services, product advisory, architecture and integration, DevOps and automation, security and compliance, and support and maintenance.',
 'source': 'https://www.fissionlabs.com/services/'}

#### Contact Information

In [36]:
get_output("contact information")

{'ANSWER': {'Email': 'info@fissionlabs.com',
  'India Phone': '+91 79955 72006',
  'US Phone': '+1 650 520 9508'},
 'source url': 'https://www.fissionlabs.com/contact-us/'}